In [33]:
print("Hello, World!")

Hello, World!


In [34]:
from langchain_groq import ChatGroq
import os

In [35]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
 GROQ_API_KEY=os.environ.get("GROQ_API_KEY")
model= ChatGroq(model="qwen/qwen3-32b", api_key=GROQ_API_KEY)

In [37]:
model=ChatGroq(model="qwen/qwen3-32b")

In [38]:
model.invoke("What is the capital of France?")

AIMessage(content='<think>\nOkay, so the user is asking, "What is the capital of France?" Hmm, let me think. I remember from school that France is a country in Europe, and its capital is a major city. Let me recall... I think it\'s Paris. Wait, isn\'t Paris known for the Eiffel Tower and the Louvre? Yeah, that sounds right. Let me double-check to make sure I\'m not mixing it up with another country. For example, Spain\'s capital is Madrid, Germany\'s is Berlin, Italy\'s is Rome. So France\'s should be Paris. I don\'t think there\'s any confusion there. Maybe they just want a straightforward answer. I should confirm if there are any other possible answers, but I\'m pretty confident it\'s Paris. Yeah, I\'ll go with Paris as the capital of France.\n</think>\n\nThe capital of France is **Paris**. Known for iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral, Paris is one of the most famous cities in the world and a major cultural, historical, and politica

In [39]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [40]:
embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [92]:
dummy=embedding_model.embed_query("What is the capital of France?")

## 1. Data INgestion

In [42]:
from langchain.document_loaders import PyPDFLoader

In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [44]:
os.path.join(os.getcwd(), "data", "sample.pdf")

'c:\\Users\\sharma.rakesh\\Desktop\\Projects\\LLM_ops\\document_portal\\notebook\\data\\sample.pdf'

In [45]:
os.getcwd()

'c:\\Users\\sharma.rakesh\\Desktop\\Projects\\LLM_ops\\document_portal\\notebook'

In [47]:
loader=PyPDFLoader(os.path.join(os.getcwd(), "data", "sample.pdf"))

In [49]:
documents=loader.load()

In [50]:
len(documents)

77

In [98]:
## there is no determnitic way to split the text,we can try different chunk sizes and overlap
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, 
                                             chunk_overlap=150,length_function=len)

In [99]:
docs=text_splitter.split_documents(documents)

In [100]:
len(docs)

765

In [101]:
docs[0].metadata

{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2023-07-20T00:30:36+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2023-07-20T00:30:36+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': 'c:\\Users\\sharma.rakesh\\Desktop\\Projects\\LLM_ops\\document_portal\\notebook\\data\\sample.pdf',
 'total_pages': 77,
 'page': 0,
 'page_label': '1'}

In [102]:
docs[0].page_content

'Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev'

In [103]:
check=embedding_model.embed_documents([docs[0].page_content])

In [104]:
len(embedding_model.embed_documents([docs[0].page_content][0]))

500

In [105]:
from langchain.vectorstores import FAISS

In [106]:
## in memory vector store
vectorstore=FAISS.from_documents(docs, embedding_model)

## 2. Retrrieval processes 

In [108]:
vectorstore.similarity_search("What is the capital of France?", k=1)

[Document(id='74c65628-b216-40a8-864e-4439b30ec680', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'c:\\Users\\sharma.rakesh\\Desktop\\Projects\\LLM_ops\\document_portal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 43, 'page_label': '44'}, page_content='Ba. Large language models are human-level prompt engineers. InThe Eleventh International Conference on\nLearning Representations, 2022.\n44')]

In [111]:
relevant_doc=vectorstore.similarity_search("What is the capital of France?", k=1)

In [112]:
relevant_doc[0]

Document(id='74c65628-b216-40a8-864e-4439b30ec680', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'c:\\Users\\sharma.rakesh\\Desktop\\Projects\\LLM_ops\\document_portal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 43, 'page_label': '44'}, page_content='Ba. Large language models are human-level prompt engineers. InThe Eleventh International Conference on\nLearning Representations, 2022.\n44')

In [113]:
print(relevant_doc[0].page_content)

Ba. Large language models are human-level prompt engineers. InThe Eleventh International Conference on
Learning Representations, 2022.
44


In [114]:
retriever=vectorstore.as_retriever(k=10)

In [115]:
retriever.invoke("llama2 finetuning benchmark experiments.")

[Document(id='7a4cd32e-8b8e-4e2b-a0e5-5839cd3c2705', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'c:\\Users\\sharma.rakesh\\Desktop\\Projects\\LLM_ops\\document_portal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 7, 'page_label': '8'}, page_content='13B 18.9 66.1 52.6 62.3 10.9 46.9 37.0 33.9\n33B 26.0 70.0 58.4 67.6 21.4 57.8 39.8 41.7\n65B 30.7 70.7 60.5 68.6 30.8 63.4 43.5 47.6\nLlama 2\n7B 16.8 63.9 48.9 61.3 14.6 45.3 32.6 29.3\n13B 24.5 66.9 55.4 65.8 28.7 54.8 39.4 39.1\n34B 27.8 69.9 58.7 68.0 24.2 62.6 44.1 43.4\n70B 37.5 71.9 63.6 69.4 35.2 68.9 51.2 54.2\nTable 3: Overall performance on grouped academic benchmarks compared to open-source base models.'),
 D

### Question: user question
### Context: based on the question retrieving the info from the vector database

In [116]:
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""

In [117]:
from langchain.prompts import PromptTemplate

In [118]:
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [119]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n        Answer the question based on the context provided below. \n        If the context does not contain sufficient information, respond with: \n        "I do not have enough information about this."\n\n        Context: {context}\n\n        Question: {question}\n\n        Answer:')

In [120]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

In [121]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [124]:
from langchain.schema.runnable import RunnablePassthrough

In [125]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [126]:
rag_chain.invoke("tell  me about the llama2 finetuning benchmark experiments?")

'<think>\nOkay, let\'s try to figure out the answer to the question about the Llama 2 fine-tuning benchmark experiments. First, I need to look at the context provided. The user mentioned a table with performance metrics for different models, including Llama 2 variants like 7B, 13B, 34B, and 70B. There\'s also a section labeled "Fine-tuned" with some numbers, and then under "ChatGPT" and "Llama 2" with various figures.\n\nThe question is asking specifically about the fine-tuning experiments for Llama 2. The context has two parts: the first part (Table 3) compares overall performance on academic benchmarks, and the second part seems to be about fine-tuned models, including ChatGPT and Llama 2 with different parameters. The numbers here are different from the first table, possibly indicating different metrics or benchmarks.\n\nLooking at the "Fine-tuned" section, there\'s a row for Llama 2 with 7B, 13B, 34B, and 70B models. The numbers here are in the range of 0.2 to 0.7, which might repr